# Implementação do Algoritmo Random Forest

---

O **Random Forest** é um dos algoritmos mais poderosos e amplamente utilizados em Machine Learning, baseado na técnica de ensemble learning. Ele combina múltiplas árvores de decisão para melhorar a estabilidade e a precisão das previsões, reduzindo a variância e mitigando o overfitting. Abaixo, descrevemos as etapas principais para sua implementação:

1. **Bootstrap (Amostragem com Reposição)**  
   - O conjunto de dados original é dividido em várias amostras bootstrap.  
   - Cada amostra bootstrap é gerada por amostragem com reposição, permitindo que certos exemplos sejam selecionados múltiplas vezes.  
   - O tamanho de cada amostra bootstrap é igual ao do conjunto de treinamento original.

2. **Feature Selection (Seleção Aleatória de Variáveis)**  
   - Diferentemente do Bagging tradicional, o Random Forest introduz aleatoriedade adicional ao selecionar um subconjunto aleatório de variáveis para cada divisão de nó em uma árvore.  
   - O número de variáveis selecionadas em cada divisão segue uma regra predefinida, geralmente a raiz quadrada do número total de variáveis no caso de classificação e um terço do total no caso de regressão.

3. **Modelagem com Árvores de Decisão (Base Learners)**  
   - Para cada amostra bootstrap, um modelo de árvore de decisão é treinado independentemente.  
   - Não há poda (pruning) das árvores, permitindo que cada uma cresça até o seu máximo potencial.

4. **Agregação das Predições (Voting ou Média)**  
   - Para tarefas de **classificação**, a agregação final ocorre por meio de uma **votação majoritária**, onde a classe mais predita entre todas as árvores se torna a saída final.  
   - Para tarefas de **regressão**, a agregação é feita calculando a **média das previsões** das árvores individuais.  

Essa abordagem garante que o **Random Forest** seja um modelo altamente eficaz, pois equilibra viés e variância de maneira mais eficiente do que um único modelo de árvore de decisão.

---

## **Diferença entre Bagging e Random Forest**

Embora o **Bagging (Bootstrap Aggregating)** e o **Random Forest** compartilhem o mesmo princípio fundamental de combinar múltiplos modelos para reduzir a variância e aumentar a estabilidade preditiva, existem diferenças cruciais entre eles:

1. **Estratégia de Modelagem:**  
   - O **Bagging** é uma abordagem geral de ensemble learning que pode ser aplicada a qualquer modelo base, como árvores de decisão, regressões lineares, entre outros.  
   - O **Random Forest**, por outro lado, é uma implementação específica do Bagging que se restringe ao uso de **árvores de decisão**, incorporando uma etapa adicional de seleção aleatória de variáveis para cada divisão de nó.

2. **Diversificação dos Modelos:**  
   - No **Bagging**, cada modelo base é treinado em um subconjunto bootstrap, mas todas as variáveis podem ser consideradas em cada divisão da árvore.  
   - No **Random Forest**, além de utilizar amostragem bootstrap, um subconjunto aleatório de variáveis é selecionado para cada nó da árvore, reduzindo ainda mais a correlação entre os modelos.

3. **Redução da Correlação entre Árvores:**  
   - O **Bagging** ajuda a reduzir a variância, mas suas árvores podem permanecer fortemente correlacionadas se todas tiverem acesso a todas as variáveis.  
   - O **Random Forest** reduz ainda mais essa correlação ao limitar o número de variáveis disponíveis para cada divisão, tornando as árvores mais independentes e aumentando a generalização do modelo.

Em resumo, o **Random Forest** pode ser visto como uma versão otimizada do **Bagging**, projetada especificamente para árvores de decisão. Essa técnica introduz maior diversidade entre os modelos individuais, levando a um desempenho geralmente superior em termos de precisão e robustez.

---

In [1]:
# Import das bibliotecas:

import numpy  as np
import pandas as pd

from sklearn.datasets        import load_iris
from sklearn.datasets        import load_diabetes

from sklearn.model_selection import train_test_split

from sklearn.tree            import DecisionTreeClassifier
from sklearn.metrics         import accuracy_score

from sklearn.tree            import DecisionTreeRegressor
from sklearn.metrics         import mean_squared_error, r2_score

In [2]:
# Exemplo da técnica Random Forest para problemas de classificação:

X = load_iris().data
y = load_iris().target

df = pd.DataFrame(X, columns=load_iris().feature_names)
df['target'] = y

def rf_classifier(df:pd.DataFrame, 
                  num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                  test_size:float=0.25
                 ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        # Feature selection
        X_train = X_train.sample(n=round(np.sqrt(X_train.shape[1])),  # Cálculo da raiz quadrada da quantidade de variáveis
                                 axis=1)
        
        y_train = df_train['target']
        
        # Modelagem (base learners)
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test[X_train.columns])})
    
    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mode(axis=1)  # Agregando o valor com maior número de aparições nas predições dos modelos
                .rename(columns={0:'y_pred'}))
 
    # Resultados
    print(model)
    print('Accuracy score:', accuracy_score(y_true=y_test, 
                                            y_pred=y_pred['y_pred']
                                           ))

    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred['y_pred'].astype(int)], 
                     axis=1)

rf_classifier(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeClassifier()
Accuracy score: 0.96


,y_test,y_pred
0,0,0
1,2,2
2,1,1
3,2,2
4,1,1
5,2,2
6,2,1
7,2,2
8,1,1
9,0,0


In [3]:
# Exemplo da técnica Random Forest para problemas de regressão:

X = load_diabetes().data
y = load_diabetes().target

df = pd.DataFrame(X, columns=load_diabetes().feature_names)
df['target'] = y

def rf_regressor(df:pd.DataFrame, 
                 num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                 test_size:float=0.25
                ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        # Feature selection
        X_train = X_train.sample(n=round(X_train.shape[1]/3),  # Cálculo da quantidade de variáveis dividida por 3
                                 axis=1)
        
        y_train = df_train['target']

        # Modelagem (base learners)
        model = DecisionTreeRegressor()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test[X_train.columns])})

    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mean(axis=1)  # Agregando as predições dos modelos baseando n a média dos resultados
                .rename('y_pred'))
 
    # Resultados
    print(model)
    print('Mean squared error:', mean_squared_error(y_true=y_test, 
                                                   y_pred=y_pred))
    print('Coefficient of determination:', r2_score(y_true=y_test, 
                                                    y_pred=y_pred))
    
    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred], 
                     axis=1)
    
rf_regressor(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeRegressor()
Mean squared error: 4199.046575342466
Coefficient of determination: 0.271689234225421


,y_test,y_pred
0,155.0,121.3
1,72.0,155.2
2,197.0,187.5
3,154.0,141.5
4,277.0,217.0
...,...,...
141,148.0,153.0
142,198.0,123.6
143,199.0,133.3
144,270.0,190.4


---

## **Conclusão**

A implementação do **Random Forest** em problemas de **classificação** e **regressão** demonstrou sua eficácia como um modelo preditivo robusto.  

### **Resultados Obtidos**
**Classificação**  
- O modelo alcançou uma **acurácia de 96%**, evidenciando que a agregação de múltiplas árvores reduziu significativamente a variância e melhorou a estabilidade da previsão.  
- O uso de seleção aleatória de variáveis garantiu que diferentes árvores explorassem padrões distintos nos dados, resultando em um desempenho superior ao de uma única árvore de decisão.

**Regressão**  
- O **Erro Quadrático Médio (MSE)** foi **4199,05**, e o coeficiente de determinação (**R²**) atingiu **0,27**.  
- Embora o desempenho tenha sido inferior ao da classificação, o modelo ainda conseguiu capturar uma parte significativa da variabilidade nos dados.  
- A otimização dos hiperparâmetros, como o número de árvores e a profundidade máxima das árvores individuais, poderia melhorar os resultados.

### **Considerações Finais**
O **Random Forest** provou ser um modelo altamente eficiente para tarefas de Machine Learning, combinando a simplicidade das árvores de decisão com a robustez dos métodos de ensemble. Sua capacidade de lidar com conjuntos de dados complexos, minimizando overfitting e maximizando a generalização, faz dele uma das abordagens mais confiáveis para modelagem preditiva.